In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [3]:
import os
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

# Optimized Image Parameters
img_size = 128  # Reduced from 256 for better performance
batch_size = 8  # Reduced from 16 for less memory usage

# Define ImageDataGenerator with minimal augmentation for better performance
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,        # Increased
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],  # Added
    fill_mode='nearest',
    validation_split=0.2
)

# Training data
train_generator = datagen.flow_from_directory(
    "C:/Users/joshu/Documents/distracted/data/imgs/train",
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=batch_size,
    subset="training",  # This will use 80% of data
    class_mode='categorical',
    seed=2020
)

# Validation data (using same generator but validation subset)
val_generator = datagen.flow_from_directory(
    "C:/Users/joshu/Documents/distracted/data/imgs/train",
    target_size=(img_size, img_size),
    color_mode='rgb',
    batch_size=batch_size,
    subset="validation",  # This will use 20% of data
    class_mode='categorical',
    seed=2020
)

# Let's verify the splits
print(f"Number of training samples: {len(train_generator.filenames)}")
print(f"Number of validation samples: {len(val_generator.filenames)}")
print(f"Classes: {train_generator.class_indices}")

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.
Number of training samples: 17943
Number of validation samples: 4481
Classes: {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


# EfficientNet (Pre-trained)

In [4]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import tensorflow as tf


# Fine tuned
# Load EfficientNetB0 with correct preprocessing
base_model = EfficientNetB0(weights='imagenet', 
                           include_top=False, 
                           input_shape=(128, 128, 3))

# Don't freeze the base model initially
base_model.trainable = True

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Increased dropout
x = Dense(10, activation='softmax')(x)

# Create model
efficientnet_model = Model(inputs=base_model.input, outputs=x)

# Use a custom learning rate schedule
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9
learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate)

# Compile with the schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)
efficientnet_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [5]:
efficientnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 128, 128, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 128, 128, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 128, 128, 3)  0           ['normalization[0][0]']      

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks for early stopping and saving the best model
#callbacks = [
    #EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),  # Stop early if validation loss doesn't improve
    #ModelCheckpoint('best_model.keras', save_best_only=True)  # Save the best model with .keras extension
#]

# Training the model
history_1 = efficientnet_model.fit(
    train_generator,  # Training data
    steps_per_epoch=train_generator.samples // batch_size,  # Number of batches of training data to run per epoch
    epochs=10,  # Adjust based on data size and performance
    validation_data=val_generator,  # Validation data
    validation_steps=val_generator.samples // batch_size,  # Number of validation steps
    callbacks=None  # Include the callbacks
)

Epoch 1/10
2242/2242 [==============================] - 1442s 639ms/step - loss: 5.7747 - accuracy: 0.4993 - val_loss: 1.5184 - val_accuracy: 0.7951
Epoch 2/10
2242/2242 [==============================] - 1473s 657ms/step - loss: 1.2512 - accuracy: 0.8096 - val_loss: 0.8346 - val_accuracy: 0.9038
Epoch 3/10
2242/2242 [==============================] - 1445s 645ms/step - loss: 0.9437 - accuracy: 0.8657 - val_loss: 0.6475 - val_accuracy: 0.9453
Epoch 4/10
2242/2242 [==============================] - 1363s 608ms/step - loss: 0.7170 - accuracy: 0.9055 - val_loss: 0.5418 - val_accuracy: 0.9480
Epoch 5/10
2242/2242 [==============================] - 1218s 543ms/step - loss: 0.5508 - accuracy: 0.9305 - val_loss: 0.4033 - val_accuracy: 0.9560
Epoch 6/10
 756/2242 [=========>....................] - ETA: 12:52 - loss: 0.4678 - accuracy: 0.9433

# ResNet50 (Pre-trained)

In [27]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Build and compile the model
resnet_model = Model(inputs=base_model.input, outputs=x)
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [28]:
resnet_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_7[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']       

# VGG16 (Pre-trained)

In [29]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Load VGG16 model pre-trained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Build and compile the model
vgg_model = Model(inputs=base_model.input, outputs=x)
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [30]:
vgg_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0   

# DenseNet

In [31]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load DenseNet model pre-trained on ImageNet
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Build and compile the model
densenet_model = Model(inputs=base_model.input, outputs=x)
densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
densenet_model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 134, 134, 3)  0          ['input_9[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d_2[0][0]']       
                                                                                            

#  MobileNetV2

In [33]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load MobileNetV2 pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='softmax')(x)

# Build and compile the model
mobilenet_model = Model(inputs=base_model.input, outputs=x)
mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 1s 0us/step


In [34]:
mobilenet_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_10[0][0]']               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']         